# 0.0 IMPORTS

In [7]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import json

In [8]:
# vai ser usado para verificar se as info que eu quero estão no arquivo que eu baixei
queries = [ "machine+learning", "data+science", "kaggle"]
# url que irei usar para biscar esses vídeos
url = "https://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"
# "search_query" vai receber a palavra que eu quero procurar dentro da variavel "query"
# "&sp" que é um código que vai ordenar os resultados do amis recente para o mais antigo (data de upload)
# "p" você não vai ver esse código no endereço da página do Youtube se for fazer uma consulta direto na plataforma, mas com 
# esse código é possível paginar a busca no Youtube.

In [9]:
# que página vou buscar? eu bisco os vídeos que ordeno pelos amis recentes
# o resultado de busca e a página do vídeo

# precisa pegar a página que retorna os resultados da busca e pegar a pa´gina de cada vídeo que é retornado.
## para fazer isso vamos usar biblioteca chamada "BeautifulSoup" e "requests"
# os datasets usados eu não tenho printis para baixar (como no kaggle). Então vou ter que criar COLETORES para as duas
# páginas no youtube.

# dica de sempre salvar a página que vc pegou com os coletores e está trabalhando no projeto. Motivos:
# - sobrecarregar o servidor se acabar testando muitas cisas várias vezes e acabar sendo bloqueado pelo site do youtube
## soluçõ: baixar todas as pag que precisamos e dados e depois pensar sobre a parte de processamento. o "requests" é uma 
## biblioteca que faz requisições no servidor de hospedagem e ele tem tanto requisições GET como POST. O "BeautifulSoup" será
## usado para fazer o parser.

# 1.0 DATA COLLECTION

In [11]:
# Criando o Coletor da Página de Busca do Youtube.

for query in queries:
    for page in range(1, 21): # para cada query eu quero que ele pegue um determinado número de páginas.
        urll = url.format( query=query, page=page ) # formata a url com sa info que eu quero. Basicamente é como se eu estivesse
        # fazendo manualemente a busca com a palavra que eu quero e salvando a quantidade de páginas que eu desejo.
        print( urll ) # faz um print para verificar como estão sendo geradas essas páginas e se está correto.
        response = rq.get( urll ) # quando se usa a tecnica GET no "requests" ele vai tentar fazer uma requisição no servidor
        # do site usando o endereço que está dentro da variavel "urll"
        
        with open( "./dados_brutos/{}_{}.html".format(query, page), 'w+' ) as output: # abro um arquivo na dentro da pasta 
            # "dados_brutos" e salvo as páginas com os nomes que eu formato usando as informações de "query" e "page" para
            # diferenciá-los. Vai salvar neste arquivo o código fonte da página que estou tentando pegar.
            output.write( response.text ) # vai transformar a página em dados string (.text) e escrever (write) no arquivo que
            # vai ter o nome já determinado acima.
        time.sleep(2) # para não sobrecarregar o servidor uso o "time.sleep" para dar uns 2 segundos de intervalo entre uma 
        # requisição e outra.

https://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=1


UnicodeEncodeError: 'charmap' codec can't encode characters in position 210476-210481: character maps to <undefined>

# 2.0 RAW DATA PROCESSING

In [ ]:
# Criando o Extrator de Dados da Página de Busca do Youtube.

# Quando executamos o código na última sessão e analisamos as páginas resultantes vemos que as infkrmações importantes para 
# pegarmos destas páginas são a príncipio o título e o link do vídeo.

for query in queries:
    for page in range(1, 21):
        with open( "./dados_brutos/{}_{}.html".format(query, page), 'r+' ) as inp:
            page_html = inp.read()
            
            parsed = bs4.BeautifulSoup( page_html )
            
            tags = parsed.findAll( "a" )
            
            for e in tags:
                if e.has_attr( "aria-describedby" ):
                    link = e[ 'href' ]
                    title = e[ 'title' ]
                    with open( "parsed_videos.json", 'a+' ) as output:
                        data = {"link": link, "" }